In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%tensorflow_version 1.x
%pip install numpy==1.19.5

In [ ]:
import tensorflow as tf
# tf.disable_v2_behavior()
import os
import sys
import numpy as np
import glob
import pandas as pd
import cv2
import argparse
from tqdm import tqdm
import random

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [ ]:
def getPatchIndices(corners_a):
  """
  For a given set of 4 corners, return it's indices inside the region as a mesh grid
  -used in unsupervised model
  """

  patch_indices = []
  for i in range(corners_a.shape[0]):
    xmin = corners_a[i][0][0]
    ymin = corners_a[i,0,1]
    xmax = corners_a[i,3,0]
    ymax = corners_a[i,3,1]
    X, Y = np.mgrid[xmin:xmax, ymin:ymax]
    patch_indices.append(np.dstack((Y,X))) 
  return np.array(patch_indices)

In [ ]:
def load_data_unsupervised(train_files_pa,train_files_pb,IA_files, points_list, batch_size, shuffle = True):

    patches_ab = []
    ca = []
    patches_b = []
    I_A = []

    for i in range(batch_size):
        index = random.randint(0, len(train_files_pa)-1) 
        patch_a = cv2.imread(train_files_pa[i], cv2.IMREAD_GRAYSCALE)
        patch_b = cv2.imread(train_files_pb[i], cv2.IMREAD_GRAYSCALE)
        image_a = cv2.imread(IA_files[i], cv2.IMREAD_GRAYSCALE)

        patch_a = np.float32(patch_a)
        patch_b = np.float32(patch_b) 
        image_a = np.float32(image_a)   

        #combine images along depth
        patch_ab = np.dstack((patch_a, patch_b))     
        corner1 = points_list[index, :, :, 0]
        
        
        patches_ab.append(patch_ab)
        ca.append(corner1)
        patches_b.append(patch_b.reshape(128, 128, 1))

        I_A.append(image_a.reshape(image_a.shape[0], image_a.shape[1], 1))

    patch_indices = getPatchIndices(np.array(ca))    
    return np.array(patches_ab), np.array(ca), np.array(patches_b), np.array(I_A), patch_indices


In [ ]:
def transformer(U, theta, out_size, name='SpatialTransformer', **kwargs):
    """Spatial Transformer Layer
    Implements a spatial transformer layer as described in [1]_.
    Based on [2]_ and edited by David Dao for Tensorflow.
    Parameters
    ----------
    U : float
        The output of a convolutional net should have the
        shape [num_batch, height, width, num_channels].
    theta: float
        The output of the
        localisation network should be [num_batch, 6].
    out_size: tuple of two ints
        The size of the output of the network (height, width)
    References
    ----------
    .. [1]  Spatial Transformer Networks
            Max Jaderberg, Karen Simonyan, Andrew Zisserman, Koray Kavukcuoglu
            Submitted on 5 Jun 2015
    .. [2]  https://github.com/skaae/transformer_network/blob/master/transformerlayer.py
    Notes
    -----
    To initialize the network to the identity transform init
    ``theta`` to :
        identity = np.array([[1., 0., 0.],
                             [0., 1., 0.]])
        identity = identity.flatten()
        theta = tf.Variable(initial_value=identity)
    """
    
    scale_h= True  

    def _repeat(x, n_repeats):
        # Process 
        # dim2 = width
        # dim1 = width*height
        # v = tf.range(num_batch)*dim1
        # print 'old v:', v # num_batch 
        # print 'new v:', tf.reshape(v, (-1, 1)) # widthx1
        # n_repeats = 20 
        # rep = tf.transpose(tf.expand_dims(tf.ones(shape=tf.stack([n_repeats, ])), 1), [1, 0]) # 1 x out_width*out_height 
        # print rep
        # rep = tf.cast(rep, 'int32')
        # v = tf.matmul(tf.reshape(v, (-1, 1)), rep) # v: num_batch x (out_width*out_height)
        # print '--final v:\n', v.eval() 
        # # v is the base. For parallel computing. 
        with tf.variable_scope('_repeat'):
            rep = tf.transpose(
                tf.expand_dims(tf.ones(shape=tf.stack([n_repeats, ])), 1), [1, 0])
            rep = tf.cast(rep, 'int32')
            x = tf.matmul(tf.reshape(x, (-1, 1)), rep)
            return tf.reshape(x, [-1])

    def _interpolate(im, x, y, out_size,scale_h):
        with tf.variable_scope('_interpolate'):
            # constants
            num_batch = tf.shape(im)[0]
            height = tf.shape(im)[1]
            width = tf.shape(im)[2]
            channels = tf.shape(im)[3]

            x = tf.cast(x, 'float32')
            y = tf.cast(y, 'float32')
            height_f = tf.cast(height, 'float32')
            width_f = tf.cast(width, 'float32')
            out_height = out_size[0]
            out_width = out_size[1]
            zero = tf.zeros([], dtype='int32')
            max_y = tf.cast(tf.shape(im)[1] - 1, 'int32')
            max_x = tf.cast(tf.shape(im)[2] - 1, 'int32')

            if scale_h:
                # scale indices from [-1, 1] to [0, width/height]
                print('--Inter- scale_h:', scale_h)
                x = (x + 1.0)*(width_f) / 2.0
                y = (y + 1.0)*(height_f) / 2.0

            # do sampling
            x0 = tf.cast(tf.floor(x), 'int32')
            x1 = x0 + 1
            y0 = tf.cast(tf.floor(y), 'int32')
            y1 = y0 + 1

            x0 = tf.clip_by_value(x0, zero, max_x)
            x1 = tf.clip_by_value(x1, zero, max_x)
            y0 = tf.clip_by_value(y0, zero, max_y)
            y1 = tf.clip_by_value(y1, zero, max_y)
            dim2 = width
            dim1 = width*height
            base = _repeat(tf.range(num_batch)*dim1, out_height*out_width)
            base_y0 = base + y0*dim2
            base_y1 = base + y1*dim2
            idx_a = base_y0 + x0
            idx_b = base_y1 + x0
            idx_c = base_y0 + x1
            idx_d = base_y1 + x1

            # use indices to lookup pixels in the flat image and restore
            # channels dim
            im_flat = tf.reshape(im, tf.stack([-1, channels]))
            im_flat = tf.cast(im_flat, 'float32')
            Ia = tf.gather(im_flat, idx_a)
            Ib = tf.gather(im_flat, idx_b)
            Ic = tf.gather(im_flat, idx_c)
            Id = tf.gather(im_flat, idx_d)

            # and finally calculate interpolated values
            x0_f = tf.cast(x0, 'float32')
            x1_f = tf.cast(x1, 'float32')
            y0_f = tf.cast(y0, 'float32')
            y1_f = tf.cast(y1, 'float32')
            wa = tf.expand_dims(((x1_f-x) * (y1_f-y)), 1)
            wb = tf.expand_dims(((x1_f-x) * (y-y0_f)), 1)
            wc = tf.expand_dims(((x-x0_f) * (y1_f-y)), 1)
            wd = tf.expand_dims(((x-x0_f) * (y-y0_f)), 1)
            output = tf.add_n([wa*Ia, wb*Ib, wc*Ic, wd*Id])
            return output

    def _meshgrid(height, width, scale_h):
        with tf.variable_scope('_meshgrid'):
            # This should be equivalent to:
            #  x_t, y_t = np.meshgrid(np.linspace(-1, 1, width),
            #                         np.linspace(-1, 1, height))
            #  ones = np.ones(np.prod(x_t.shape))
            #  grid = np.vstack([x_t.flatten(), y_t.flatten(), ones])
            #  Process: 
            # x1 = tf.ones(shape=tf.stack([height, 1])) # ones matrix of size height x 1 
            # x2 =  tf.expand_dims(tf.linspace(0.0, tf.cast(width,'float32'), width), 1) # linespace(0,width,width) then expanded one more 
            #                                 # dimesion to width x 1 
            # x3 = tf.transpose(x2, [1, 0])  # 1 x 1 x width 
            # x_t = tf.matmul(x1, x3)   # 1x height x width
            # y_t = tf.matmul(tf.expand_dims(tf.linspace(0.0, tf.cast(height,'float32'), height), 1),
            #             tf.ones(shape=tf.stack([1, width])))

            # x_t_flat = tf.reshape(x_t, (1, -1)) # [linespace(0,width,width),  linespace(0,width,width) ....] 
            #  return: [-1, .. 0 ... , 1]
            #          [-1, .. 0 ... , 1]
            #          [1, .. 1 ... , 1]

            if scale_h:
                x_t = tf.matmul(tf.ones(shape=tf.stack([height, 1])),
                                tf.transpose(tf.expand_dims(tf.linspace(-1.0, 1.0, width), 1), [1, 0]))
                y_t = tf.matmul(tf.expand_dims(tf.linspace(-1.0, 1.0, height), 1),
                                tf.ones(shape=tf.stack([1, width])))
            else: 
                x_t = tf.matmul(tf.ones(shape=tf.stack([height, 1])),
                            tf.transpose(tf.expand_dims(tf.linspace(0.0, tf.cast(width,'float32'), width), 1), [1, 0]))
                y_t = tf.matmul(tf.expand_dims(tf.linspace(0.0, tf.cast(height,'float32'), height), 1),
                            tf.ones(shape=tf.stack([1, width])))

            x_t_flat = tf.reshape(x_t, (1, -1))
            y_t_flat = tf.reshape(y_t, (1, -1))

            ones = tf.ones_like(x_t_flat)
            grid = tf.concat([x_t_flat, y_t_flat, ones], 0)
            # sess = tf.get_default_session()
            # print '--grid: \n', grid.eval() # (session=sess.as_default())
            return grid

    def _transform(theta, input_dim, out_size, scale_h):
        with tf.variable_scope('_transform'):
            num_batch = tf.shape(input_dim)[0]
            height = tf.shape(input_dim)[1]
            width = tf.shape(input_dim)[2]
            num_channels = tf.shape(input_dim)[3]
            #  Changed 
            # theta = tf.reshape(theta, (-1, 2, 3))
            theta = tf.reshape(theta, (-1, 3, 3))
            theta = tf.cast(theta, 'float32')

            #  Added: add two matrices M and B defined as follows in 
            # order to perform the equation: H x M x [xs...;ys...;1s...] + H x [width/2...;height/2...;0...]
            theta_shape = theta.get_shape().as_list()
            # initial




            # grid of (x_t, y_t, 1), eq (1) in ref [1]
            height_f = tf.cast(height, 'float32')
            width_f = tf.cast(width, 'float32')
            out_height = out_size[0]
            out_width = out_size[1]
            grid = _meshgrid(out_height, out_width, scale_h)
            grid = tf.expand_dims(grid, 0)
            grid = tf.reshape(grid, [-1])
            grid = tf.tile(grid, tf.stack([num_batch])) # stack num_batch grids 
            grid = tf.reshape(grid, tf.stack([num_batch, 3, -1]))

            # Transform A x (x_t, y_t, 1)^T -> (x_s, y_s)
            T_g = tf.matmul(theta, grid)
            x_s = tf.slice(T_g, [0, 0, 0], [-1, 1, -1])
            # Ty changed 
            # y_s = tf.slice(T_g, [0, 1, 0], [-1, 1, -1])
            y_s = tf.slice(T_g, [0, 1, 0], [-1, 1, -1])
            # Ty added 
            t_s = tf.slice(T_g, [0, 2, 0], [-1, 1, -1])
            # The problem may be here as a general homo does not preserve the parallelism 
            # while an affine transformation preserves it. 
            t_s_flat = tf.reshape(t_s, [-1])

            # Avoid zero division 
            zero = tf.constant(0, dtype=tf.float32)
            one = tf.constant(1, dtype=tf.float32)


            # smaller
            small = tf.constant(1e-7, dtype=tf.float32)
            smallers = 1e-6*(one - tf.cast(tf.greater_equal(tf.abs(t_s_flat), small),tf.float32)) 
             
           
            t_s_flat = t_s_flat + smallers 
            condition =  tf.reduce_sum(tf.cast(tf.greater(tf.abs(t_s_flat), small),tf.float32))
            # Ty changed 
            # x_s_flat = tf.reshape(x_s, [-1])
            # y_s_flat = tf.reshape(y_s, [-1])
            x_s_flat = tf.reshape(x_s, [-1])/t_s_flat
            y_s_flat = tf.reshape(y_s, [-1])/t_s_flat
            

            input_transformed =  _interpolate( input_dim, x_s_flat, y_s_flat,out_size,scale_h)

            output = tf.reshape(
                input_transformed, tf.stack([num_batch, out_height, out_width, num_channels]))
            return output, condition

    with tf.variable_scope(name):
        output,condition = _transform(theta, U, out_size, scale_h)
        return output, condition


def batch_transformer(U, thetas, out_size, name='BatchSpatialTransformer'):
    """Batch Spatial Transformer Layer
    Parameters
    ----------
    U : float
        tensor of inputs [num_batch,height,width,num_channels]
    thetas : float
        a set of transformations for each input [num_batch,num_transforms,6]
    out_size : int
        the size of the output [out_height,out_width]
    Returns: float
        Tensor of size [num_batch*num_transforms,out_height,out_width,num_channels]
    """
    with tf.variable_scope(name):
        num_batch, num_transforms = map(int, thetas.get_shape().as_list()[:2])
        indices = [[i]*num_transforms for i in xrange(num_batch)]
        input_repeated = tf.gather(U, tf.reshape(indices, [-1]))
        return transformer(input_repeated, thetas, out_size)

In [ ]:
# Auxiliary matrices used to solve DLT
Aux_M1  = np.array([
          [ 0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [ 1 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [ 0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [ 0 , 0 , 1 , 0  , 0 , 0 , 0 , 0 ],
          [ 0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [ 0 , 0 , 0 , 0  , 1 , 0 , 0 , 0 ],
          [ 0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [ 0 , 0 , 0 , 0  , 0 , 0 , 1 , 0 ]], dtype=np.float64)

Aux_M2  = np.array([
          [ 0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [ 0 , 1 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [ 0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [ 0 , 0 , 0 , 1  , 0 , 0 , 0 , 0 ],
          [ 0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [ 0 , 0 , 0 , 0  , 0 , 1 , 0 , 0 ],
          [ 0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [ 0 , 0 , 0 , 0  , 0 , 0 , 0 , 1 ]], dtype=np.float64)

Aux_M3  = np.array([
          [0],
          [1],
          [0],
          [1],
          [0],
          [1],
          [0],
          [1]], dtype=np.float64)



Aux_M4  = np.array([
          [-1 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 ,-1 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  ,-1 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 0 ,-1 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ]], dtype=np.float64)


Aux_M5  = np.array([
          [0 ,-1 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 ,-1  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 ,-1 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 0 ,-1 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ]], dtype=np.float64)

Aux_M6  = np.array([
          [-1 ],
          [ 0 ],
          [-1 ],
          [ 0 ],
          [-1 ],
          [ 0 ],
          [-1 ],
          [ 0 ]], dtype=np.float64)

Aux_M71 = np.array([
          [0 , 1 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [1 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 1  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 1 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 1 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 1 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 0 , 1 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 1 , 0 ]], dtype=np.float64)

Aux_M72 = np.array([
          [1 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [-1 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 1 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 ,-1 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 1 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  ,-1 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 1 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 0 ,-1 , 0 ]], dtype=np.float64)

Aux_M8  = np.array([
          [0 , 1 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [0 ,-1 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 1  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 ,-1  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 1 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 ,-1 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 0 , 1 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 0 ,-1 ]], dtype=np.float64)
Aux_Mb  = np.array([
          [0 ,-1 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [1 , 0 , 0 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 , -1  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 1 , 0  , 0 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 ,-1 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 1 , 0 , 0 , 0 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 0 ,-1 ],
          [0 , 0 , 0 , 0  , 0 , 0 , 1 , 0 ]], dtype=np.float64)


In [ ]:
def TensorDLT(H4pt, C4A , MiniBatchSize):
    
    pts_1_tile = tf.expand_dims(C4A, [2]) # BATCH_SIZE x 8 x 1
    
    # Solve for H using DLT
    pred_h4p_tile = tf.expand_dims(H4pt, [2]) # BATCH_SIZE x 8 x 1
    # 4 points on the second image
    pred_pts_2_tile = tf.add(pred_h4p_tile, pts_1_tile)


    # Auxiliary tensors used to create Ax = b equation
    M1_tile = tf.tile(tf.expand_dims(tf.constant(Aux_M1,tf.float32),[0]),[MiniBatchSize,1,1])
    M2_tile = tf.tile(tf.expand_dims(tf.constant(Aux_M2,tf.float32),[0]),[MiniBatchSize,1,1])
    M3_tile = tf.tile(tf.expand_dims(tf.constant(Aux_M3,tf.float32),[0]),[MiniBatchSize,1,1])
    M4_tile = tf.tile(tf.expand_dims(tf.constant(Aux_M4,tf.float32),[0]),[MiniBatchSize,1,1])
    M5_tile = tf.tile(tf.expand_dims(tf.constant(Aux_M5,tf.float32),[0]),[MiniBatchSize,1,1])
    M6_tile = tf.tile(tf.expand_dims(tf.constant(Aux_M6,tf.float32),[0]),[MiniBatchSize,1,1])
    M71_tile = tf.tile(tf.expand_dims(tf.constant(Aux_M71,tf.float32),[0]),[MiniBatchSize,1,1])
    M72_tile = tf.tile(tf.expand_dims(tf.constant(Aux_M72,tf.float32),[0]),[MiniBatchSize,1,1])
    M8_tile = tf.tile(tf.expand_dims(tf.constant(Aux_M8,tf.float32),[0]),[MiniBatchSize,1,1])
    Mb_tile = tf.tile(tf.expand_dims(tf.constant(Aux_Mb,tf.float32),[0]),[MiniBatchSize,1,1])

    # Form the equations Ax = b to compute H
    # Form A matrix
    A1 = tf.matmul(M1_tile, pts_1_tile) # Column 1
    A2 = tf.matmul(M2_tile, pts_1_tile) # Column 2
    A3 = M3_tile                   # Column 3
    A4 = tf.matmul(M4_tile, pts_1_tile) # Column 4
    A5 = tf.matmul(M5_tile, pts_1_tile) # Column 5
    A6 = M6_tile                   # Column 6
    A7 = tf.matmul(M71_tile, pred_pts_2_tile) *  tf.matmul(M72_tile, pts_1_tile)# Column 7
    A8 = tf.matmul(M71_tile, pred_pts_2_tile) *  tf.matmul(M8_tile, pts_1_tile)# Column 8

    A_mat = tf.transpose(tf.stack([tf.reshape(A1,[-1,8]),tf.reshape(A2,[-1,8]),\
                                   tf.reshape(A3,[-1,8]),tf.reshape(A4,[-1,8]),\
                                   tf.reshape(A5,[-1,8]),tf.reshape(A6,[-1,8]),\
                                   tf.reshape(A7,[-1,8]),tf.reshape(A8,[-1,8])],axis=1), perm=[0,2,1]) # BATCH_SIZE x 8 (A_i) x 8


    print('--Shape of A_mat:', A_mat.get_shape().as_list())
    # Form b matrix
    b_mat = tf.matmul(Mb_tile, pred_pts_2_tile)
    print('--shape of b:', b_mat.get_shape().as_list())

    # Solve the Ax = b
    H_8el = tf.matrix_solve(A_mat , b_mat)  # BATCH_SIZE x 8.
    print('--shape of H_8el', H_8el)


    # Add ones to the last cols to reconstruct H for computing reprojection error
    h_ones = tf.ones([MiniBatchSize, 1, 1])
    H_9el = tf.concat([H_8el,h_ones],1)
    H_flat = tf.reshape(H_9el, [-1,9])
    H_mat = tf.reshape(H_flat,[-1,3,3])   # BATCH_SIZE x 3 x 3

    return H_mat

In [ ]:
def homography_model(img, image_size, batch_size):
  x = tf.layers.conv2d(inputs=img, name='Conv2D1', padding='same',filters=64, kernel_size=[3,3], activation=None)
  x = tf.layers.batch_normalization(x, name='BatchNorm1')
  x = tf.nn.relu(x, name='Relu1')

  x = tf.layers.conv2d(inputs=x, name='Conv2D2', padding='same',filters=64, kernel_size=[3,3], activation=None) 
  x = tf.layers.batch_normalization(x, name='BatchNorm2')
  x = tf.nn.relu(x, name='Relu2')

  x = tf.layers.max_pooling2d(inputs=x, pool_size=[2,2], strides=2)

  x = tf.layers.conv2d(inputs=x, name='Conv2D3', padding='same',filters=64, kernel_size=[3,3], activation=None)
  x = tf.layers.batch_normalization(x, name='BatchNorm3')
  x = tf.nn.relu(x, name='Relu3')

  x = tf.layers.conv2d(inputs=x, name='Conv2D4', padding='same',filters=64, kernel_size=[3,3], activation=None)
  x = tf.layers.batch_normalization(x, name='BatchNorm4')
  x = tf.nn.relu(x, name='Relu4')

  x = tf.layers.max_pooling2d(inputs=x, pool_size=[2,2], strides=2)

  x = tf.layers.conv2d(inputs=x, name='Conv2D5', padding='same',filters=128, kernel_size=[3,3], activation=None)
  x = tf.layers.batch_normalization(x, name='BatchNorm5')
  x = tf.nn.relu(x, name='Relu5')

  x = tf.layers.conv2d(inputs=x, name='Conv2D6', padding='same',filters=128, kernel_size=[3,3], activation=None)
  x = tf.layers.batch_normalization(x, name='BatchNorm6')
  x = tf.nn.relu(x, name='Relu6')

  x = tf.layers.max_pooling2d(inputs=x, pool_size=[2,2], strides=2)

  x = tf.layers.conv2d(inputs=x, name='Conv2D7', padding='same',filters=128, kernel_size=[3,3], activation=None)
  x = tf.layers.batch_normalization(x, name='BatchNorm7')
  x = tf.nn.relu(x, name='Relu7')

  x = tf.layers.conv2d(inputs=x, name='Conv2D8', padding='same',filters=128, kernel_size=[3,3], activation=None)
  x = tf.layers.batch_normalization(x, name='BatchNorm8')
  x = tf.nn.relu(x, name='Relu8')

  #flattening layer
  x = tf.contrib.layers.flatten(x)

  #Fully-connected layers
  x = tf.layers.dense(inputs=x, name='FC1',units=1024, activation=tf.nn.relu)
  x = tf.layers.dropout(x,rate=0.5, training=True,name='Dropout')
  x = tf.layers.batch_normalization(x, name='BatchNorm9')
  H4 = tf.layers.dense(inputs=x, name='FCfinal',units=8, activation=None)

  return H4

In [ ]:
def unsupervised_HomographyNet(patch_batches, c_a,  p_b, i_a, patch_indices, batch_size =50 ) :

    batch_size,h,w,channels = i_a.get_shape().as_list()

    H4_batches = homography_model(patch_batches, [128,128,1],50) # H4 = [dx1,dy1,dx2,dy2,dx3,dy3,dx4,dy4] 

    corners_a = tf.reshape(c_a,[batch_size,8]) # convert to 8x1 [x1,y1,x2,y2,x3,y3,x4,y4]

    H_batches = TensorDLT(H4_batches , corners_a, batch_size)
    
    # compute M
    M = np.array([[w/2.0, 0., w/2.0],
                  [0., h/2.0, h/2.0],
                  [0., 0., 1.]]).astype(np.float32)
    
    tensor_M = tf.constant(M, tf.float32)
    tensor_M = tf.expand_dims(tensor_M, [0])
    M_batches   = tf.tile(tensor_M, [batch_size, 1,1]) #make 'batch_size' number of copies. 
    
    #compute M_inv
    M_inv = np.linalg.inv(M)
    tensor_M_inv = tf.constant(M_inv, tf.float32)
    tensor_M_inv = tf.expand_dims(tensor_M_inv, [0])
    M_inv_batches   = tf.tile(tensor_M_inv, [batch_size,1,1]) #make 'batch_size' number of copies.
    
    H_scaled = tf.matmul(tf.matmul(M_inv_batches, H_batches), M_batches)

#     Pa = tf.slice(patch_batches,[0,0,0,0],[batch_size,128,128,1])

    warped_Ia, _ = transformer(i_a, H_scaled, (h,w))     

    warped_Ia = tf.reshape(warped_Ia, [batch_size, h,w])
    warped_Pa = tf.gather_nd(warped_Ia, patch_indices, name=None, batch_dims=1)

    warped_Pa = tf.transpose(warped_Pa, perm = [0,2,1])
    
    warped_Pa = tf.reshape(warped_Pa, [batch_size, 128, 128, 1])
    
    return warped_Pa, p_b, H_batches

In [ ]:
def TrainOperation(PatchPairsPH, CornerPH, Patch2PH, Images1PH, patchIndicesPH, train_files_pa, train_files_pb, image_files, pointsList, NumTrainSamples, ImageSize,NumEpochs, batch_size, SaveCheckPoint, CheckPointPath, LatestFile,LogsPath):
    """
    Inputs: 
    ImgPH is the Input Image placeholder
    LabelPH is the one-hot encoded label placeholder
    DirNamesTrain - Variable with Subfolder paths to train files
    TrainLabels - Labels corresponding to Train/Test
    NumTrainSamples - length(Train)
    ImageSize - Size of the image
    NumEpochs - Number of passes through the Train data
    MiniBatchSize is the size of the MiniBatch
    SaveCheckPoint - Save checkpoint every SaveCheckPoint iteration in every epoch, checkpoint saved automatically after every epoch
    CheckPointPath - Path to save checkpoints/model
    DivTrain - Divide the data by this number for Epoch calculation, use if you have a lot of dataor for debugging code
    LatestFile - Latest checkpointfile to continue training
    BasePath - Path to COCO folder without "/" at the end
    LogsPath - Path to save Tensorboard Logs
	ModelType - Supervised or Unsupervised Model
    Outputs:
    Saves Trained network in CheckPointPath and Logs to LogsPath
    """      
    # Predict output with forward pass
    patchb_pred, patchb_true, _ = unsupervised_HomographyNet(PatchPairsPH, CornerPH, Patch2PH, Images1PH,patchIndicesPH, batch_size)

    with tf.name_scope('Loss'):
        loss =  tf.reduce_mean(tf.abs(patchb_pred - patchb_true))

    with tf.name_scope('Adam'):
        Optimizer = tf.train.AdamOptimizer(learning_rate=1e-5).minimize(loss)

    epoch_loss_ph = tf.placeholder(tf.float32, shape=None)
    loss_summary = tf.summary.scalar('Loss_per_iteration', loss)
    epoch_loss_summary = tf.summary.scalar('loss_per_epoch', epoch_loss_ph)
    # tf.summary.image('Anything you want', AnyImg)

    # Merge all summaries into a single operation
    MergedSummaryOP1 = tf.summary.merge([loss_summary])
    MergedSummaryOP2 = tf.summary.merge([epoch_loss_summary])
    # MergedSummaryOP = tf.summary.merge_all()

    # Setup Saver
    Saver = tf.train.Saver()
    AccOverEpochs=np.array([0,0])

    with tf.Session() as sess:  

        if LatestFile is not None:
            Saver.restore(sess, CheckPointPath + LatestFile + '.ckpt')
          # Extract only numbers from the name
            StartEpoch = int(''.join(c for c in LatestFile.split('a')[0] if c.isdigit()))
            print('Loaded latest checkpoint with the name ' + LatestFile + '....')
        else:
            sess.run(tf.global_variables_initializer())
            StartEpoch = 0
            print('New model initialized....')

        # Tensorboard
        Writer = tf.summary.FileWriter(LogsPath, graph=tf.get_default_graph())
        
        L1_loss = []
        for Epochs in tqdm(range(StartEpoch, NumEpochs)):

            NumIterationsPerEpoch = int(NumTrainSamples/batch_size)
            Loss=[]
            epoch_loss=0

            for PerEpochCounter in tqdm(range(NumIterationsPerEpoch)):

                PatchPairsBatch, Corner1Batch, patch2Batch, Image1Batch, patchIndicesBatch = load_data_unsupervised(train_files_pa,train_files_pb,image_files, pointsList, batch_size, shuffle = True)
                FeedDict = {PatchPairsPH: PatchPairsBatch, CornerPH: Corner1Batch, Patch2PH: patch2Batch, Images1PH: Image1Batch, patchIndicesPH: patchIndicesBatch}

                _, LossThisBatch, Summary = sess.run([Optimizer, loss, MergedSummaryOP1], feed_dict=FeedDict)
                Loss.append(LossThisBatch)
                epoch_loss = epoch_loss + LossThisBatch

          # Tensorboard
            Writer.add_summary(Summary, Epochs*NumIterationsPerEpoch + PerEpochCounter)
            epoch_loss = epoch_loss/NumIterationsPerEpoch
            print('Completed epoch: ', Epochs)
            print("Printing Epoch loss:  ",  np.mean(Loss), "\n")
            L1_loss.append(np.mean(Loss))
            SaveName = CheckPointPath + str(Epochs) + 'model.ckpt'
            Saver.save(sess, save_path=SaveName)
            print('\n' + SaveName + ' Model Saved...')
            Summary_epoch = sess.run(MergedSummaryOP2,feed_dict={epoch_loss_ph: epoch_loss})
            Writer.add_summary(Summary_epoch,Epochs)
            Writer.flush()

        np.savetxt(LogsPath + "losshistory_unsupervised.txt", np.array(L1_loss), delimiter = ",")

            


In [ ]:
tf.compat.v1.reset_default_graph()

In [ ]:
ModelType = 'Unsup'

NumEpochs = 75
DivTrain = 1
MiniBatchSize = 1
LoadCheckPoint = 0
# Parser.add_argument('--LogsPath', default='../Logs/', help='Path to save Logs for Tensorboard, Default=Logs/')

DivTrain = float(DivTrain)
batch_size = 50
CheckPointPath = '/content/drive/MyDrive/Project_1/unsupervised_checkpoints/'
LogsPath = '/content/drive/MyDrive/Project_1/unsupervised_logs/'
ImageSize = [128,128,1]
SaveCheckPoint = True

if(not (os.path.isdir(LogsPath))):
    print(LogsPath, "  was not present, creating the folder...")
    os.makedirs(LogsPath)
    
print(" UnSupervised  Model Training using Tensorflow ...")

if(not (os.path.isdir(CheckPointPath))):
    print(CheckPointPath, "  was not present, creating the folder...")
    os.makedirs(CheckPointPath)

# files_in_dir, SaveCheckPoint, ImageSize, NumTrainSamples, _ = SetupAll(BasePath, CheckPointPath)
train_files_PA = glob.glob( '/content/drive/MyDrive/Project_1/S_train/P_A'+ '/*.jpg')
train_files_PB = glob.glob( '/content/drive/MyDrive/Project_1/S_train/P_B'+ '/*.jpg')
image_files = glob.glob( '/content/drive/MyDrive/Project_1/S_train/I_A'+ '/*.jpg')
NumTrainSamples = len(train_files_PA)

print("Number of Training Samples:..", NumTrainSamples)

pointsList = np.load('/content/drive/MyDrive/Project_1/S_train/points_list.npy')

CornerPH = tf.placeholder(tf.float32, shape=(batch_size, 4,2))
PatchPairsPH = tf.placeholder(tf.float32, shape=(batch_size, 128, 128 ,2))
Patch2PH = tf.placeholder(tf.float32, shape=(batch_size, 128, 128, 1))
Images1PH = tf.placeholder(tf.float32, shape=(batch_size, 240, 320, 1))
patchIndicesPH = tf.placeholder(tf.int32, shape=(batch_size, 128, 128 ,2))

LatestFile = None

TrainOperation(PatchPairsPH, CornerPH, Patch2PH, Images1PH, patchIndicesPH, train_files_PA, train_files_PB, image_files, pointsList, NumTrainSamples, ImageSize,NumEpochs, batch_size, SaveCheckPoint, CheckPointPath, LatestFile,LogsPath)


/content/drive/MyDrive/Project_1/unsupervised_logs/   was not present, creating the folder...
 UnSupervised  Model Training using Tensorflow ...
/content/drive/MyDrive/Project_1/unsupervised_checkpoints/   was not present, creating the folder...
Number of Training Samples:.. 5000
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)


100%|██████████| 100/100 [02:32<00:00,  1.52s/it]


Completed epoch:  0
Printing Epoch loss:   64.17118 



  1%|▏         | 1/75 [02:33<3:09:49, 153.91s/it]


/content/drive/MyDrive/Project_1/unsupervised_checkpoints/0model.ckpt Model Saved...


...


/content/drive/MyDrive/Project_1/unsupervised_checkpoints/73model.ckpt Model Saved...



100%|██████████| 100/100 [01:49<00:00,  1.10s/it]


Completed epoch:  74
Printing Epoch loss:   56.153854 



100%|██████████| 75/75 [2:21:00<00:00, 112.81s/it]


/content/drive/MyDrive/Project_1/unsupervised_checkpoints/74model.ckpt Model Saved...
